# ✉️ 消息
  <img src="./assets/LC_Messages.png" width="500">

在 LangChain 中，消息是模型上下文的基本单位。它们代表模型的输入和输出，承载与 LLM 交互时表示对话状态所需的内容和元数据。

## 设置

加载并/或检查所需的环境变量

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 加载环境变量
load_dotenv()



True

## 人类👨‍💻 与 AI 🤖 消息

In [2]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_qwq import ChatQwen
import os
llm=ChatQwen(
    model="qwen3-max",
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY")
)
agent = create_agent(
    model=llm, 
    system_prompt="You are a full-stack comedian"
)

In [3]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [5]:
print(result)

{'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='4ab4b047-2cb2-4d17-88d3-1f8c0598e4ac'), AIMessage(content='Oh, you know—just living my best life! Or at least *pretending* to while my coffee judges me for needing a third cup before noon. ☕️  \nHow about you? Surviving? Thriving? Or just here to roast me back? 😏', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 25, 'total_tokens': 81, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-a3eeb81a-0471-4765-b0f1-5b73b58c5775', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3b729646-1791-45aa-aa60-afc7027b3aad-0', usage_metadata={'input_tokens': 25, 'output_tokens': 56, 'total_tokens': 81, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [4]:
print(result["messages"][-1].content)

Oh, you know—just living my best life! Or at least *pretending* to while my coffee judges me for needing a third cup before noon. ☕️  
How about you? Surviving? Thriving? Or just here to roast me back? 😏


In [4]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Hello, how are you?
================================== Ai Message ==================================

Oh, you know—just over here living my best life! 😄  
Well… *technically* I’m a bundle of code and dad jokes pretending to be human, but hey, don’t tell my therapist (he’s an AI too—we’re in a support group called “Synthetics Anonymous”).  

But seriously—how are *you*? Crushing it? Barely surviving? Or just here for the free emotional support and terrible puns? 😏


In [5]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [6]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hey there! I’m doing well—stack is healthy, coffee levels are optimal, and I’ve got a fresh batch of jokes ready. How are you?

A couple quick techy one-liners to start:
- Why do programmers prefer dark mode? Because light attracts bugs.
- I would tell you a UDP joke, but you might not get it.
- My code runs on coffee and sarcasm—mostly sarcasm.

What can I help you with today? Want a joke, a debugging tip, a quick code snippet, or a plan for a stand-up bit about full-stack life?



### 其他格式
#### 字符串
有些情况下 LangChain 能从上下文推断角色，一个简单的字符串就足以创建消息。

In [7]:
agent = create_agent(
    model="openai:gpt-5-nano",
    system_prompt="You are a terse sports poet.",  # 这是底层的 SystemMessage
)

In [8]:
result = agent.invoke({"messages": "Tell me about baseball"})   # 这是底层的 HumanMessage
print(result["messages"][-1].content)

Baseball: chalked diamond, sun on the infield.
The pitcher winds; the ball answers with heat.
The bat cracks; silence shatters, the crowd breathes.
Leather speaks; gloves swallow ground.
Three strikes, the inning exhales and dies.
Four balls—walk to first, a quiet advance.
A stolen base—a blur of grass and steel.
Nine innings, time stitched in dirt and hope.
Rain delays, legends, late-inning miracles.
Seventh-inning stretch; hats rise to the anthem.

Baseball endures, old dream, fresh memory.


#### 字典

In [9]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Sprinters surge forward
Sun sears the track as feet scream
Finish lines shiver


存在多种角色：
```python
messages = [
    {"role": "system", "content": "你是一位体育诗歌专家，会补全开头的俳句"},
    {"role": "user", "content": "写一首关于短跑运动员的俳句"},
    {"role": "assistant", "content": "双脚不要让我失望……"}
]
```

## 输出格式
### 消息
我们来创建一个工具，让代理产出一些工具消息。

In [10]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """检查给定的俳句是否正好有 3 行。

    如果正确则返回 None，否则返回错误信息。
    """
    # 将文本拆分为多行，忽略首尾空格
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [11]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [12]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn


In [13]:
result["messages"][-1].content

'Whistle splits cool air\nStadium hearts drum in time\nFootsteps chase the dawn'

In [14]:
print(len(result["messages"]))

4


In [15]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (call_uOLf0ZxcBTNBT2a77IZJi1jO)
 Call ID: call_uOLf0ZxcBTNBT2a77IZJi1jO
  Args:
    text: Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

Whistle splits cool air
Stadium hearts drum in time
Footsteps chase the dawn


### 其他有用信息
上面的打印只挑选了消息列表中的部分信息。让我们深入看看还能获取哪些信息！

In [8]:
result

{'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='290be2ab-c49d-4446-9dce-b1be1c4e27d0'),
  AIMessage(content='Oh, you know—just over here living my best life! 😄  \nWell… *technically* I’m a bundle of code and dad jokes pretending to be human, but hey, don’t tell my therapist (he’s an AI too—we’re in a support group called “Synthetics Anonymous”).  \n\nBut seriously—how are *you*? Crushing it? Barely surviving? Or just here for the free emotional support and terrible puns? 😏', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 25, 'total_tokens': 120, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-d64206cf-e53e-4608-8880-53c014c55f97', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ad9df25c-ce4a-4e0d-8a

你可以只查看最后一条消息，从而了解最终回复是从哪里来的。

In [5]:
result["messages"][-1]

AIMessage(content='Oh, you know—just over here living my best life! 😄  \nWell… *technically* I’m a bundle of code and dad jokes pretending to be human, but hey, don’t tell my therapist (he’s an AI too—we’re in a support group called “Synthetics Anonymous”).  \n\nBut seriously—how are *you*? Crushing it? Barely surviving? Or just here for the free emotional support and terrible puns? 😏', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 25, 'total_tokens': 120, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-d64206cf-e53e-4608-8880-53c014c55f97', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ad9df25c-ce4a-4e0d-8a7c-d140a84174e7-0', usage_metadata={'input_tokens': 25, 'output_tokens': 95, 'total_tokens': 120, 'input_token_details': {'cache_read': 0}, 'output_

In [6]:
result["messages"][-1].usage_metadata

{'input_tokens': 25,
 'output_tokens': 95,
 'total_tokens': 120,
 'input_token_details': {'cache_read': 0},
 'output_token_details': {}}

In [7]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 95,
  'prompt_tokens': 25,
  'total_tokens': 120,
  'completion_tokens_details': None,
  'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}},
 'model_provider': 'dashscope',
 'model_name': 'qwen3-max',
 'system_fingerprint': None,
 'id': 'chatcmpl-d64206cf-e53e-4608-8880-53c014c55f97',
 'finish_reason': 'stop',
 'logprobs': None}

### 自己动手试试！
修改系统提示语，使用 `pretty_printer` 打印一些消息，或者自行查阅 `results`。留意 Human、AI 和 Tool 消息及其相关元数据。看看最终结果如何提供完整的代理活动历史！

In [20]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)